In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns
from tensorflow import keras
import random
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split, KFold

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
to_test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv").drop("id", axis = 1)

# pseudo labeling

In [ ]:
fake = pd.read_csv("../input/tps08-nn-file/7.87122.csv")
test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")
test["loss"] = fake.loss
train = pd.concat([train, test]).reset_index(drop = True)

In [ ]:
y = train.loss
X = train.drop(["id", "loss"], axis = 1)

# Check the distribution of the target value

In [ ]:
sns.distplot(y, hist=True, kde=False, 
             bins=int(180/5), color = 'blue',
             hist_kws={'edgecolor':'black'})

In [ ]:
def cmlp(dr, hu1,hu2, hu3, hu4, hu5, hu6, hu7, hu8, hu9, lr):
    HIDDEN_LAYER_1 = [hu1, hu2]    #[256, 256]
    HIDDEN_LAYER_2 = [hu3, hu4, hu5]    #[160, 160, 160]
    HIDDEN_LAYER_3 = [hu6, hu7, hu8, hu9]    #[128, 128, 128, 128]
    TARGET_NUM = 1 
    input = tf.keras.layers.Input(shape=(X.shape[1], ))
    #part_1
    x1 = tf.keras.layers.BatchNormalization()(input)
    x1 = tf.keras.layers.Dropout(dr)(x1)
    for units in HIDDEN_LAYER_1:
        x1 = tf.keras.layers.Dense(units)(x1)
        x1 = tf.keras.layers.BatchNormalization()(x1)
        x1 = tf.keras.layers.Activation(tf.keras.activations.swish)(x1)
        x1 = tf.keras.layers.Dropout(dr)(x1)
    # part_2
    x2 = tf.keras.layers.BatchNormalization()(input)
    x2 = tf.keras.layers.Dropout(dr)(x2)
    for units in HIDDEN_LAYER_2:
        x2 = tf.keras.layers.Dense(units)(x2)
        x2 = tf.keras.layers.BatchNormalization()(x2)
        x2 = tf.keras.layers.Activation(tf.keras.activations.swish)(x2)
        x2 = tf.keras.layers.Dropout(dr)(x2)
    # part_3
    x3 = tf.keras.layers.BatchNormalization()(input)
    x3 = tf.keras.layers.Dropout(dr)(x3)
    for units in HIDDEN_LAYER_3:
        x3 = tf.keras.layers.Dense(units)(x3)
        x3 = tf.keras.layers.BatchNormalization()(x3)
        x3 = tf.keras.layers.Activation(tf.keras.activations.swish)(x3)
        x3 = tf.keras.layers.Dropout(dr)(x3)
    x = tf.keras.layers.concatenate([x1, x2, x3])
    output = tf.keras.layers.Dense(TARGET_NUM)(x)
    model = tf.keras.models.Model(inputs=input, outputs=output)
    model.compile(
        optimizer = tf.optimizers.Adam(learning_rate = lr),
        metrics   = [tf.keras.metrics.RootMeanSquaredError()],
        loss      = "MSE"
    )
    return model
def lm(path):
    model = tf.keras.models.load_model(path) 
    return model
from sklearn.metrics import mean_squared_error
def my_metrics(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)
def set_seed(seed = 2021):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    return random_state

# Config

In [ ]:
folds = 7
seed_list = [20]

# Tuning

In [ ]:
def objective(trial):
    params = {
        'dr' : trial.suggest_uniform('dr' ,0.01 , 0.3),
        'hu1': trial.suggest_int('hu1' ,220 , 300),
        'hu2': trial.suggest_int('hu2' ,230 , 300),
        'hu3': trial.suggest_int('hu3' ,145 , 155),
        'hu4': trial.suggest_int('hu4' ,135 , 150),
        'hu5': trial.suggest_int('hu5' ,135 , 150),
        'hu6': trial.suggest_int('hu6' ,110 , 120),
        'hu7': trial.suggest_int('hu7' ,80 , 100),
        'hu8': trial.suggest_int('hu8' ,80 , 100),
        'hu9': trial.suggest_int('hu9' ,96 , 113),
        'lr' : trial.suggest_uniform('lr' , 0.002 , 0.004)
    }
    score = 0
    for seed in seed_list: 
        kf = KFold(n_splits = folds ,random_state= seed, shuffle = True)
        fn = 1
        for idx_train,idx_test in kf.split(X, y):
            X_train,X_test=X.iloc[idx_train],X.iloc[idx_test]
            y_train,y_test=y.iloc[idx_train],y.iloc[idx_test]
            cp = tf.keras.callbacks.ModelCheckpoint(f'./model{fn}.h5', save_best_only = True, monitor = 'val_root_mean_squared_error', mode = 'min')
            es = EarlyStopping(monitor = 'val_root_mean_squared_error', patience = 25, mode = 'min')
            pc = optuna.integration.TFKerasPruningCallback(trial, 'val_root_mean_squared_error')
            model = cmlp(**params) 
            set_seed(seed)
            model.fit(X_train ,  y_train, validation_data = (X_test, y_test), epochs = 300, 
                       batch_size = 2000, callbacks = [cp, es
                                                      , pc
                                            ] , verbose = 0 )
            model = lm(f'./model{fn}.h5')
            y_pred = model.predict(X_test).ravel()
            fn += 1
            score += my_metrics(y_test.values.ravel(), y_pred)                
        del model
    return score / 7
import optuna
study = optuna.create_study(direction = 'minimize' , study_name = 'nn'
                         #   , pruner = optuna.pruners.HyperbandPruner()
                           )
study.optimize(objective , n_trials = 220)
print('numbers of the finished trials:' , len(study.trials))
print('the best params:' , study.best_trial.params)
print('the best value:' , study.best_value)
print("done")
#time.sleep(60)

In [ ]:
pcmlp = {'dr': 0.17815556906426416, 'hu1': 278, 'hu2': 251, 
         'hu3': 155, 'hu4': 141, 'hu5': 141, 'hu6': 111, 
         'hu7': 88, 'hu8': 80, 'hu9': 104, 
         'lr': 0.0038937535106663996} # Best is trial 2 with value: 6.259858319677164.

In [ ]:
pcmlp = {'dr': 0.2714603811596955, 'hu1': 227, 'hu2': 233, 
         'hu3': 149, 'hu4': 139, 'hu5': 137, 'hu6': 113, 'hu7': 96, 'hu8': 96, 
         'hu9': 103, 'lr': 0.0029735538680568385} # the best value: 7.893019035495328

In [ ]:
score = 0
for seed in seed_list: 
    set_seed(seed)
    print("seed", seed)
    kf = KFold(n_splits = folds, random_state= seed, shuffle = True)
    fn = 1
    for idx_train,idx_test in kf.split(X, y):
        X_train,X_test=X.iloc[idx_train],X.iloc[idx_test]
        y_train,y_test=y.iloc[idx_train],y.iloc[idx_test]
        cp = tf.keras.callbacks.ModelCheckpoint(f'./3 layers regression_fold_{fn}_seed_{seed}.h5', 
                            save_best_only = True, monitor = 'val_root_mean_squared_error', mode = 'min')
        es = EarlyStopping(monitor = 'val_root_mean_squared_error', patience = 50, mode = 'min')
        model = cmlp(**pcmlp) 
        model.fit(X_train ,  y_train, validation_data = (X_test, y_test), epochs = 300, 
                   batch_size = 2000, callbacks = [cp, es], verbose = 1 )
        model = lm(f'./3 layers regression_fold_{fn}_seed_{seed}.h5')
        y_pred = model.predict(X_test).ravel()
        score_temp = my_metrics(y_test.values.ravel(), y_pred)  
        print("fold", fn, "model", "Validation score", score_temp)
        score += score_temp
        fn += 1
print("Mean CV score", (score / 7)/ len(seed_list))

In [ ]:
import glob

In [ ]:
output = []  
for filepath in glob.iglob('./*.h5'): # Mean CV score 7.892253827629668
    model = lm(filepath)
    pred = model.predict(to_test).ravel()
    output.append(pred)
    del model
    del pred
y_pred = sum(output) / len(output)

In [ ]:
final_pred = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")
final_pred.loss = y_pred
final_pred

In [ ]:
final_pred.to_csv('submission.csv',index=False)